# Modelos de IA para previsão de notas acadêmicas

Com base em um conjunto de dados de notas acadêmicas, vamos criar modelos de aprendizado de máquina para prever as notas dos alunos e alertar sobre riscos de possíveis dificultades como prova final e reprovações.

## 1. Tratamento dos Dados

Primeiramente, precisamos adaptar os dados para o formato adequado para o treinamento dos modelos. No conjunto de dados `notas.csv`, Nos interessamos apenas nas colunas `AlunoID` que corresponde ao identificador do aluno (precisamos ter seu perfil salvo para melhor análise da IA), `Disciplina` que é a disciplina em questão e as colunas `Unidade X` sendo X o número da unidade.


Vamos começar importando as bibliotecas, abrir o CSV e convertendo as notas pra float:


In [23]:
import pandas as pd


# Carrega os dados do arquivo CSV
notas = pd.read_csv('dados/notas.csv')

# Converter as colunas de notas para o tipo float
for unidade in ['Unidade 1', 'Unidade 2', 'Unidade 3', 'Unidade 4', 'Unidade 5']:
    notas[unidade] = notas[unidade].str.replace(',', '.').astype(float)

Após isso, vamos criar uma coluna `Media` que será a média das notas de cada aluno em cada disciplina. Não vamos considerar a nota da prova final para o cálculo da média, pois ela é uma nota de recuperação e vamos considerar apenas as unidades que tenham notas pois, se estiver em branco, significa que a disciplina não tem nota naquela unidade.


In [24]:
# Calcular a média das notas, sem prova final
notas['Media'] = notas[['Unidade 1', 'Unidade 2', 'Unidade 3', 'Unidade 4', 'Unidade 5']].mean(axis=1, skipna=True)


Com a coluna `Media` criada, agora podemos pivotear os dados. `AlunoID` será o índice, `Disciplina` será as colunas e as notas serão os valores. Isso nos dará um DataFrame onde cada linha representa um aluno e cada coluna uma disciplina, com as notas correspondentes. Um aluno pode ter cursado uma disciplina mais de uma vez em caso de reprovação, por isso pode gerar conflitos no pivotamento. Para resolver isso, vamos usar a menor nota, pois é a nota que o aluno pode ter reprovado/ficado de prova final.


In [25]:
notas_pivoteadas = notas.pivot_table(
    index='AlunoID',
    columns='Disciplina',
    values='Media',
    aggfunc='min'
)

O resultado ficando assim:


In [26]:
notas_pivoteadas.head()

Disciplina,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,BANCOS DE DADOS RELACIONAIS,CIRCUITOS DIGITAIS,COMPILADORES,COMPUTAÇÃO GRÁFICA,CÁLCULO DIFERENCIAL E INTEGRAL I,CÁLCULO DIFERENCIAL E INTEGRAL II,ELETRICIDADE E MAGNETISMO,EMPREENDEDORISMO,...,TRABLO DE CONCLUSÃO DE CURSO II - TCC II,TÓPICOS EM ARQUITETURA DE COMPUTADORES,TÓPICOS EM BIOINFORMÁTICA,TÓPICOS EM COMPUTAÇÃO APLICADA,TÓPICOS EM ENGENHARIA DE SOFTWARE,TÓPICOS EM PROGRAMAÇÃO,TÓPICOS EM REDES DE COMPUTADORES,TÓPICOS EM SISTEMAS COMPUTACIONAIS,VISÃO COMPUTACIONAL,ÁLGEBRA LINEAR
AlunoID,,,,,,,,,,,,,,,,,,,,,
1,2.333333,9.666667,NaN,9.033333,7.333333,8.333333,9.333333,9.266667,8.333333,NaN,...,NaN,NaN,8.333333,NaN,8.633333,NaN,NaN,NaN,NaN,7.500000
2,7.066667,9.166667,8.500000,8.533333,7.033333,7.166667,7.100000,6.233333,NaN,NaN,...,NaN,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,8.433333,7.000000
3,7.000000,9.833333,9.333333,7.500000,7.266667,8.700000,6.333333,8.133333,NaN,9.066667,...,NaN,7.0,5.100000,8.566667,8.333333,NaN,9.866667,NaN,NaN,8.266667
4,7.466667,8.666667,7.100000,7.433333,7.100000,6.433333,8.633333,7.466667,NaN,NaN,...,NaN,NaN,10.000000,NaN,7.866667,6.200000,9.966667,NaN,6.500000,2.500000
5,NaN,NaN,NaN,9.933333,7.866667,5.666667,8.766667,NaN,NaN,NaN,...,NaN,5.4,6.400000,NaN,8.366667,6.666667,NaN,NaN,6.433333,NaN


Agora, vamos salvar o DataFrame resultante em um arquivo CSV.


In [27]:
notas_pivoteadas.to_csv('dados/notas_pivoteadas.csv', index=False)

## 2. Treinamenro do Modelo

In [28]:
import pandas as pd

df = pd.read_csv('dados/notas_pivoteadas.csv')

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

x = df[
[    'PROGRAMAÇÃO ESTRUTURADA',
    'INTRODUÇÃO À LÓGICA',
    'INGLÊS TÉCNICO E CIENTÍFICO',
    'INFORMÁTICA E SOCIEDADE',
    'CÁLCULO DIFERENCIAL E INTEGRAL I',
    'INTRODUÇÃO À METODOLOGIA CIENTÍFICA']
]
y = df[
[    'CÁLCULO DIFERENCIAL E INTEGRAL II',
    'CIRCUITOS DIGITAIS',
    'ESTRUTURAS DE DADOS',
    'LABORATÓRIO DE PROGRAMAÇÃO',
    'MATEMÁTICA DISCRETA',
    'PROGRAMAÇÃO ORIENTADA A OBJETOS',]
]

dados = pd.concat([x, y], axis=1).dropna()
x = dados[x.columns]
y = dados[y.columns]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
mean_absolute_error(y_test, y_pred)


0.6429111111111138

In [30]:
# ver precisao e notas reais
for i, col in enumerate(y.columns):
    print(f'Precisão para {col}: {mean_absolute_error(y_test[col], y_pred[:, i])}')
    print(f'Notas reais: {y_test[col].values[:5]}')
    print(f'Notas previ: {y_pred[:, i][:5]}')
    print()

Precisão para CÁLCULO DIFERENCIAL E INTEGRAL II: 0.6468000000000019
Notas reais: [9.26666667 8.13333333 8.5        6.23333333 8.13333333]
Notas previ: [8.79566667 8.13333333 8.66833333 8.828      8.13333333]

Precisão para CIRCUITOS DIGITAIS: 0.2750666666666678
Notas reais: [9.03333333 7.5        9.83333333 8.53333333 7.5       ]
Notas previ: [9.07233333 7.5        8.88833333 8.92466667 7.5       ]

Precisão para ESTRUTURAS DE DADOS: 0.651333333333338
Notas reais: [8.9        8.03333333 8.93333333 5.96666667 8.03333333]
Notas previ: [8.98166667 8.03333333 8.48766667 8.696      8.03333333]

Precisão para LABORATÓRIO DE PROGRAMAÇÃO: 0.6862666666666734
Notas reais: [9.43333333 9.2        9.43333333 7.8        9.2       ]
Notas previ: [8.66033333 9.2        7.451      8.476      9.2       ]

Precisão para MATEMÁTICA DISCRETA: 0.0736666666666638
Notas reais: [7.33333333 7.         7.33333333 7.66666667 7.        ]
Notas previ: [7.31166667 7.         7.32       7.33333333 7.        ]

Precis

In [31]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.multioutput import MultiOutputRegressor

# ...carregue e prepare x e y como já está no seu código...

model = MultiOutputRegressor(RandomForestRegressor())

# KFold com 5 divisões
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# cross_val_score retorna scores negativos para erro, por isso usamos 'neg_mean_absolute_error'
scores = cross_val_score(model, x, y, cv=kf, scoring='neg_mean_absolute_error')

# Como os scores são negativos, inverta o sinal para obter o MAE positivo
mae_scores = -scores

print(f'MAE em cada fold: {mae_scores}')
print(f'MAE médio: {mae_scores.mean():.2f}')

MAE em cada fold: [0.68528889 0.68474444 0.090375   0.40331944 0.41186111]
MAE médio: 0.46


In [32]:
x = df[[
    'PROGRAMAÇÃO ESTRUTURADA',
    'INTRODUÇÃO À LÓGICA',
    'INGLÊS TÉCNICO E CIENTÍFICO',
    'INFORMÁTICA E SOCIEDADE',
    'CÁLCULO DIFERENCIAL E INTEGRAL I',
    'INTRODUÇÃO À METODOLOGIA CIENTÍFICA',
    'CÁLCULO DIFERENCIAL E INTEGRAL II',
    'CIRCUITOS DIGITAIS',
    'ESTRUTURAS DE DADOS',
    'LABORATÓRIO DE PROGRAMAÇÃO',
    'MATEMÁTICA DISCRETA',
    'PROGRAMAÇÃO ORIENTADA A OBJETOS'
]]

y = df[[
    'ÁLGEBRA LINEAR',
    'ARQUITETURA DE COMPUTADORES',
    'PROBABILIDADE E ESTATÍSTICA',
    'BANCO DE DADOS',
    'INTERFACE HUMANO COMPUTADOR',
    'PROCESSAMENTO DIGITAL DE IMAGENS',
]]

dados = pd.concat([x, y], axis=1).dropna()
x = dados[x.columns]
y = dados[y.columns]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
mean_absolute_error(y_test, y_pred)

0.599333333333336

In [33]:
# ver precisao e notas reais
for i, col in enumerate(y.columns):
    print(f'Precisão para {col}: {mean_absolute_error(y_test[col], y_pred[:, i])}')
    print(f'Notas reais: {y_test[col].values[:5]}')
    print(f'Notas previ: {y_pred[:, i][:5]}')
    print()

Precisão para ÁLGEBRA LINEAR: 0.5281333333333393
Notas reais: [7.5        8.26666667 7.16666667 7.         8.26666667]
Notas previ: [5.734      8.26666667 7.76733333 6.726      8.26666667]

Precisão para ARQUITETURA DE COMPUTADORES: 1.875533333333334
Notas reais: [2.33333333 7.         2.33333333 7.06666667 7.        ]
Notas previ: [6.229      7.         6.883      6.13433333 7.        ]

Precisão para PROBABILIDADE E ESTATÍSTICA: 0.6804666666666671
Notas reais: [9.1        5.8        8.76666667 7.         5.8       ]
Notas previ: [8.21066667 5.8        6.367      7.11333333 5.8       ]

Precisão para BANCO DE DADOS: 0.1180000000000053
Notas reais: [9.66666667 9.83333333 9.66666667 9.16666667 9.83333333]
Notas previ: [9.51833333 9.83333333 9.66333333 9.605      9.83333333]

Precisão para INTERFACE HUMANO COMPUTADOR: 0.2769333333333336
Notas reais: [9.7        8.03333333 9.66666667 8.5        8.03333333]
Notas previ: [9.405      8.03333333 9.24233333 9.16533333 8.03333333]

Precisão par